In [ ]:
# Imports
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)

from src import *
import scipy.sparse.linalg as spsl
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
# Algorithm parameters
max_energy_level = 3
kd_ratio = 2.5
noise_thresholds = 1/(10**np.arange(1,7,.5))
epsilon_values = np.array([delta/10 for delta in noise_thresholds])
K = 500
delta_t = 0.39
num_trials = 20

# Hamiltonian Parameters
molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.595'
basis_set = 'sto-3g'
mapper = 'parity'

In [ ]:
# Generate Hamiltonian and get true eigenenergies
hamiltonian = molecular_hamiltonian(molecule,basis_set,mapper)
sparse_hamiltonian = hamiltonian.to_matrix(sparse = True)
num_qubits = hamiltonian.num_qubits

v, w = spsl.eigsh(sparse_hamiltonian,k=20,which = 'SA')
true_eigenenergies = np.unique(np.round(v,8))[:max_energy_level+1]

In [ ]:
# Construct reference state
indices = np.argsort(sparse_hamiltonian.diagonal())
reference_state = bitstring_superposition_state(num_qubits,[bin(indices[i])[2:] for i in [0,1,1,2,3,4,5]])

# Get evolved reference states
d = int((K)/kd_ratio)
time_evolution_operator = -1j*sparse_hamiltonian*delta_t
evolved_reference_states = spsl.expm_multiply(time_evolution_operator,reference_state,start=0,stop=d+K+1,num = d+K+2)

In [ ]:
# Get MODMD observables
pauli_strings, coeffs = zip(*sorted(hamiltonian.to_list(), key = lambda x: np.abs(np.real(x[1])), reverse = True))

modmd_observables = ([SparsePauliOp('I' * num_qubits).to_matrix(sparse=True)] + 
                     [SparsePauliOp(p).to_matrix(sparse=True) for p in pauli_strings[60:66]])

In [ ]:
# ODMD Results
odmd_results = {epsilon:[] for epsilon in epsilon_values}
odmd_observables = [SparsePauliOp('I' * num_qubits).to_matrix(sparse=True)]

X_elements = generate_X_elements(odmd_observables,d,K,reference_state,evolved_reference_states)

for i, epsilon in enumerate(epsilon_values):
    
    for trial in range(num_trials):

        gaussian_noise = np.random.normal(0,epsilon,size=X_elements.shape) + 1j * np.random.normal(0,epsilon,size=X_elements.shape)
        noisy_X_elements = X_elements + gaussian_noise

        odmd_results[epsilon].append(
            modmd_eigenenergies(len(odmd_observables),noise_thresholds[i],noisy_X_elements,delta_t,K,kd_ratio,max_energy_level))
        odmd_results[epsilon][-1] = np.append(odmd_results[epsilon][-1],[-np.inf] * (max_energy_level + 1 - len(odmd_results[epsilon][-1])))

In [ ]:
# MODMD Results
modmd_results = {epsilon:[] for epsilon in epsilon_values}
X_elements = generate_X_elements(modmd_observables,d,K,reference_state,evolved_reference_states)

for i,epsilon in enumerate(epsilon_values):
    
    for trial in range(num_trials):

        gaussian_noise = np.random.normal(0,epsilon,size=X_elements.shape) + 1j * np.random.normal(0,epsilon,size=X_elements.shape)
        noisy_X_elements = X_elements + gaussian_noise

        modmd_results[epsilon].append(
            modmd_eigenenergies(len(modmd_observables),noise_thresholds[i],noisy_X_elements,delta_t,K,kd_ratio,max_energy_level))
        modmd_results[epsilon][-1] = np.append(modmd_results[epsilon][-1],[-np.inf] * (max_energy_level + 1 - len(modmd_results[epsilon][-1])))

In [ ]:
# Compute errors
absolute_odmd_errors = {epsilon: np.array([np.abs(odmd_results[epsilon][i] - true_eigenenergies) 
                        for i in range(num_trials)]) for epsilon in epsilon_values}
absolute_modmd_errors = {epsilon: np.array([np.abs(modmd_results[epsilon][i] - true_eigenenergies) 
                        for i in range(num_trials)]) for epsilon in epsilon_values}

odmd_errors_average = np.array([np.average(absolute_odmd_errors[epsilon],0) for epsilon in epsilon_values])
modmd_errors_average = np.array([np.average(absolute_modmd_errors[epsilon],0) for epsilon in epsilon_values])

In [ ]:
# Plotting
colors = get_color_set('LiH')
labels = [r'$|\delta E_0|$',r'$|\delta E_1|$',r'$|\delta E_2|$',r'$|\delta E_3|$']
matplotlib.rcParams.update({'font.size': 14})

for energy_level in range(max_energy_level+1):
    plt.loglog(1/epsilon_values,odmd_errors_average[:,energy_level], '--x', color = colors[energy_level])
    plt.loglog(1/epsilon_values,modmd_errors_average[:,energy_level], '-o', color = colors[energy_level], label = labels[energy_level])

plt.xlabel(r'1/$\epsilon_\text{noise}$')
plt.ylabel('Absolute Error')
plt.legend(framealpha = 0, fontsize = 14)